In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 

use_finetuned_model = True  

# Define model paths
finetuned_model_path = "fine_tuned_model"
qwen_model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Select model
model_name = finetuned_model_path if use_finetuned_model else qwen_model_name
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tok = AutoTokenizer.from_pretrained(model_name)

# Define stop tokens
if use_finetuned_model:
    stop_token_ids = tok("<|im_end|>", return_tensors="pt")["input_ids"]
else:
    stop_token_ids = tok("<|endoftext|>", return_tensors="pt")["input_ids"]

# Prompt setup
prompt = "How many instances of the letter ‘r’ are in the word ‘raspberry’?"
prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"

inputs = tok(prompt, return_tensors="pt").to(model.device)

max_new_tokens = 200
generated_ids = inputs["input_ids"].clone()

for _ in range(max_new_tokens):
    outputs = model(input_ids=generated_ids)
    next_token_id = torch.argmax(outputs.logits[:, -1, :], dim=-1).unsqueeze(0)
    generated_ids = torch.cat([generated_ids, next_token_id], dim=1)
    
    token_text = tok.decode(next_token_id[0], skip_special_tokens=True)
    print(token_text, end="", flush=True)

    # Stop if generated stop token
    if next_token_id[0].item() in stop_token_ids:
        break

print("\nDone.")


The word "raspberry" contains 3 instances of the letter 'r'.

Done.
